In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import time
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D,MaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential,load_model
from keras.utils import to_categorical
import keras
from keras.layers import InputLayer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from keras.layers import InputLayer

In [ ]:
data_path='/content/drive/My Drive/dataset/mask_dataset'
data_dir_list=os.listdir(data_path)

In [ ]:
img_data_list = []
labels = []
CATEGORIES=["with_mask","without_mask"]
def read_data():
  for dataset in CATEGORIES:
    path=os.path.join(data_path,dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    class_num=CATEGORIES.index(dataset) 
    for img in os.listdir(path):
      input_img=cv2.imread(os.path.join(path,img))
    
      input_img=cv2.resize(input_img,(224,224))
     
      img_data_list.append(input_img)
      labels.append(class_num)
  img_data = np.array(img_data_list)
  return img_data, labels

X,Y=read_data()

Loaded the images of dataset-with_mask

Loaded the images of dataset-without_mask



In [ ]:
img_x=np.array(X)
img_x = img_x.astype('float32')
img_x = img_x/255
img_x.shape

(3833, 224, 224, 3)

In [ ]:
labels=np.array(Y)
num_classes=2
labels =to_categorical(labels, num_classes)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(img_x, 
                                                    labels, 
                                                    test_size=0.2, 
                                                    stratify=np.array(labels))
                                                    
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_CHANNELS=3
input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT,IMAGE_CHANNELS)
x_train=x_train.reshape(x_train.shape[0],IMAGE_WIDTH,IMAGE_HEIGHT,3)
x_test=x_test.reshape(x_test.shape[0],IMAGE_WIDTH,IMAGE_HEIGHT,3)
labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [ ]:
from keras.applications import ResNet50
input_shape=(224,224,3)
resnet = ResNet50(include_top=True, weights='imagenet')

out = Dense(2, activation='softmax', name='predictions')(resnet.layers[-2].output)
resnet_model = Model(resnet.input, out)

for layer in resnet_model.layers:
  layer.trainable = True

pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in resnet_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # This is added back by InteractiveShellApp.init_path()


,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f142a417250>,input_2,True
1,<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7f14203157d0>,conv1_pad,True
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f1340160f10>,conv1_conv,True
3,<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f1340149dd0>,conv1_bn,True
4,<tensorflow.python.keras.layers.core.Activation object at 0x7f134017b990>,conv1_relu,True
...,...,...,...
172,<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f1150f03ad0>,conv5_block3_3_bn,True
173,<tensorflow.python.keras.layers.merge.Add object at 0x7f1150f0c390>,conv5_block3_add,True
174,<tensorflow.python.keras.layers.core.Activation object at 0x7f1150ea46d0>,conv5_block3_out,True
175,<tensorflow.python.keras.layers.pooling.GlobalAveragePooling2D object at 0x7f1150ea4750>,avg_pool,True


In [ ]:
resnet_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
from keras.callbacks import ModelCheckpoint
resnet_model.compile(Adam(lr=.001), loss='categorical_crossentropy', metrics=['accuracy']) 
BATCH_SIZE=30
epochs=20
model_name="best_model"
model_dir='/content/drive/My Drive/dataset/'
checkpoint_callback = ModelCheckpoint(model_dir + model_name+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
resnet_model.fit(x_train, y_train,epochs=epochs, validation_data=(x_test, y_test),batch_size=BATCH_SIZE,callbacks=[checkpoint_callback])

Epoch 1/20
103/103 [==============================] - 24s 185ms/step - loss: 0.0337 - accuracy: 0.9919 - val_loss: 0.1822 - val_accuracy: 0.9687

Epoch 00001: val_loss improved from inf to 0.18224, saving model to /content/drive/My Drive/dataset/best_model.h5
Epoch 2/20
103/103 [==============================] - 18s 177ms/step - loss: 0.0224 - accuracy: 0.9924 - val_loss: 0.2385 - val_accuracy: 0.9283

Epoch 00002: val_loss did not improve from 0.18224
Epoch 3/20
103/103 [==============================] - 18s 177ms/step - loss: 0.0149 - accuracy: 0.9954 - val_loss: 0.4021 - val_accuracy: 0.9622

Epoch 00003: val_loss did not improve from 0.18224
Epoch 4/20
103/103 [==============================] - 18s 177ms/step - loss: 0.0169 - accuracy: 0.9949 - val_loss: 0.1560 - val_accuracy: 0.9687

Epoch 00004: val_loss improved from 0.18224 to 0.15602, saving model to /content/drive/My Drive/dataset/best_model.h5
Epoch 5/20
103/103 [==============================] - 18s 177ms/step - loss: 0.010